# Reinforcement Machine Learnig


제어특강 마지막 차시에서 배운 강화학습의 내용을 정리하는 것을 목표로 한다.

## 1. 강화학습 기본 개념
![RL](images/RL_example.png)

controller: 행동(action, 𝑢)을 결정하는 역할을 한다.

reward function: 행동에 대한 성과를 평가하여 보상(reward, r)을 제공한다.

state, $x$: 환경이 현재 어떤 상태에 있는지 나타냄

process: state 및 action과 상호작용하며 일어나는 시스템의 동적 변화를 의미 

강화학습의 흐름
```
1. Controller는 현재 상태 (𝑥)를 기반으로 행동(𝑢)을 선택
2. Process는 action (𝑢)을 반영하여 새로운 상태(𝑥)로 변환
3. Reward function이 해당 action (𝑢)에 대한 보상(𝑟)을 제공
4. Controller는 이 정보를 학습하여 더 나은 행동을 하도록 개선
```
강화학습의 특성
```
Optimal (최적화): 최적의 정책을 찾는 과정
Adaptive (적응형): 환경의 변화에 적응할 수 있음
Model-free (모델 프리): 환경 모델 없이 학습 가능
```

## 2. Markov Decision Process
**강화학습에서 다음 상태$x^t$는 현재 상태 $x$와 행동 $u$에 의해 결정됨 이는 입력과 상태에 따라 다음 상태가 정해지는 이산상태방정식 모델돠 매우 유사함!!**

## 3. 예시 문제를 통한 개념 설명
![RL_2](images/RL_example_2.png)
![RL_3](images/RL_example_3.png)


1차원 공간에서 베터리 충전과 쓰레기를 줍는 목표를 가진 로봇 문제를 통해 강화학습의 주요 개념들을 설명하겠다.



### 3.1 Transition Function
로봇의 다음 상태 ($x^t$)는 현재 상태 ($x$)와 action ($u$)에 의해 결정된다. 다음 상태와 현재 상태, action을 연결하는 함수가 전이함수이다.

$$
x^t = f(x, u) =
\begin{cases}
x, & \text{if } x \text{ is terminal (0 or 5)} \\
x + u, & \text{otherwise}
\end{cases}
$$
로봇이 0, 5에 도달하면 움직이지 않도록 전이함수가 설계되었다. 


### 3.2 reward function

만약 현재 상태에서 어떤 액션을 취했을 때, 다음 상태가 목적을 달성하는 방향이면 보상값을 주는 것이 보상함수이다.
$$
r = \rho(x, u) =
\begin{cases}
1, & \text{if } x = 1 \text{ and } u = -1 \text{ (power pack)} \\
5, & \text{if } x = 4 \text{ and } u = 1 \text{ (trash)} \\
0, & \text{otherwise}
\end{cases}
$$
배터리나 쓰레기에 도달하면 보상을 주도록 설계

### 3.3 Policy $\pi$ 

현재 상태 $x$와 action $u$를 매핑하는 함수이다.


다음과 같은 $\pi$를 예시로 들 수 있다.
$$
\pi(0) = *  \quad \pi(1) = -1  \quad \pi(2) = 1  \quad \pi(3) = 1  \quad \pi(4) = 1  \quad \pi(5) = *
$$
터미널에선 행동을 주지 않고, 1, 2, 3, 4 상태에서의 행동을 매핑하고 있다.


## Learning Goal

보상이 최대가 되는 정책 $\pi$를 찾는 것이 강화학습의 목표이다.

$R^\pi (x_0)$는 초기 상태 $x_0$에서 정책 $\pi$를 따랐을 때 기대할 수 있는 총보상이다.

현재 상태 $x_k$와 그 상태에서의 action, 즉 $\pi(x_k)$에 대한 보상함수 $\rho$에 discount factor, $\gamma^k$를 곱한값을 해당 정책에서 가질 수 있는 모든 상태에 대해 더한 값으로 정의될 수 있다.
$$
R^\pi (x_0) = \sum_{k=0}^{\infty} \gamma^k r_{k+1} = \sum_{k=0}^{\infty} \gamma^k \rho(x_k, \pi(x_k))
$$

$\gamma$는 0에서 1사이의 값을 가지며 그 역할은 다음과 같다.
```
1. 무한 합을 유한 값으로 바운딩하여 수렴성을 보장
2. 미래에 대한 불확실성을 반영 (미래 보상의 가치를 낮춤)
```

위의 예시처럼 이론적으로 잘 정의된 시스템에서는 모든 상태와 행동에 대한 보상을 동등하게 고려하여 $\gamma$값을 모두 1로 두고 푸는 것이 좋겠지만 

**실제 환경에서는 미래에 대한 보상이 확실하지 않고, 정책이 가질 수 있는 상태와 action의 개수가 무한에 가깝게 많을 수 있다.(따라서 pseudo horizon 설정 필요)**

**따라서 $\gamma$값을 0.9~0.99정도로 설정하여 미래에 대한 보상을 점진적으로 낮게 고려하고 무한합이 수렴할 수 있도록 만드는 것이 일반적이다.**

## Value Function, Bellman Equation
value function에는 두가지 유형이 있다.

### (1) V-Function (state value function)

특정 상태 $x_0$에서 정책 $\pi$을 따를 때 기대되는 누적 보상으로서 다음과 같이 정의할 수 있다.
$$
V^\pi (x_0) = R^\pi (x_0)
$$
---
### (2) Q-Function (state-action value function)

특정 상태 $x_0$에서 특정 action $u_0$을 수행한 후, 정책 $\pi$을 따를 때 기대되는 누적 보상으로서 다음과 같이 정의할 수 있다.
$$
Q^\pi (x_0, u_0) = \rho(x_0, u_0) + \gamma R^\pi (x_1)
$$
Q-function의 유도과정은 다음과 같다.
$$
R^\pi (x_0) = \sum_{k=0}^{\infty} \gamma^k \rho(x_k, \pi(x_k))
= \rho(x_0, \pi(x_0)) + \gamma \sum_{k=0}^{\infty} \gamma^k \rho(x_{k+1}, \pi(x_{k+1}))
$$

$$
= \rho(x_0, \pi(x_0)) + \gamma R^\pi (x_1)
$$
따라서 $x_0$에서 action $u_0$를 따른다고 하면 최종적으로 다음과 같이 유도된다.
$$
Q^\pi (x_0, u_0) = \rho(x_0, u_0) + \gamma R^\pi (x_1)
$$

**여기서 첫 번째 action, $u_0$는 정책 $\pi$와 무관하게 자유롭게 선택된다.**

---
### (3) Bellman Equation

Q-function은 Bellman Equation꼴로 유도될 수 있다.
$$
Q^\pi (x_0, u_0) = \rho(x_0, u_0) + \gamma [\rho(x_1, \pi(x_1)) + \gamma R^\pi (x_2)]
$$

$$
= \rho(x_0, u_0) + \gamma Q^\pi (x_1, \pi(x_1))
$$

최종적으로 얻어지는 Q-function의 벨만 방정식 표현법은 다음과 같다. ($f$는 전이함수)
$$
Q^\pi (x, u) = \rho(x, u) + \gamma Q^\pi (f(x, u), \pi(f(x, u)))
$$
$$
=\rho(x, u) + \gamma V^\pi (x')
$$
optimal Q-function은 다음과 같이 정의될 수 있다. 
$$Q^*= \max_{\pi}Q^{\pi}$$




## Q-Value Iteration
**Q값이 최대가 되게 하는 $\pi$를 찾는 것이 핵심이다.** 

$\pi$가 상태와 액션을 매핑하므로 다음 Q값 생성 가능 

$\gamma$값에 따라 다음 상태의 V-function 값에 대한 가중치는 점점 낮아짐 -> Q가 수렴할때까지 반복!

어떤 정책 $\pi$에 대한 Q값을 계산할 수 있음!

이제 최적화문제를 풀면 $\gamma$에 의해 설계된 pseudo horizon까지의 $\pi$ 최적화 완료!!